In [113]:
# CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

import boto3
import s3fs
import fastparquet
import awswrangler as wr
import os

import pyarrow.parquet as pq
import pyarrow as pa

import requests
import json
from io import BytesIO
from pandas.tseries.offsets import BDay

import math

In [115]:
# INICIALIZAMOS UNA SESION EN AWS
session = boto3.Session(
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
    region_name=os.environ['S3_REGION'])

s3_client = boto3.client('s3')
s3 = boto3.resource('s3')

In [116]:
bucket_name = 'stock-market-historical-data'
processed_folder = 'processed/marketstack/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name

In [117]:
def download_s3_parquet_file(s3, bucket, key):
    buffer = BytesIO()
    s3.Object(bucket, key).download_fileobj(buffer)
    return buffer

In [118]:
# CARGAMOS EL DATAFRAME CON TODOS LOS DATOS DE CIERRE DE TODOS LOS ACTIVOS DESDE 02/01/1991
processed_folder = 'processed/marketstack/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name
marketstack_historical_data = pd.DataFrame(pq.read_table(download_s3_parquet_file(s3, bucket_name,full_path)).to_pandas())

In [119]:
# CARGAMOS EL DATAFRAME CON TODOS LOS DATOS DE CIERRE DE TODOS LOS ACTIVOS DESDE 02/01/1991
processed_folder = 'processed/yfinance/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name
yfinance_historical_data = pd.DataFrame(pq.read_table(download_s3_parquet_file(s3, bucket_name,full_path)).to_pandas())

In [120]:
# CARGAMOS EL DATAFRAME CON TODOS LOS DATOS DE CIERRE DE TODOS LOS ACTIVOS DESDE 02/01/1991
processed_folder = 'processed/bme-data/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name
bme_historical_data = pd.DataFrame(pq.read_table(download_s3_parquet_file(s3, bucket_name,full_path)).to_pandas())
bme_historical_data.drop('OHL', axis=1, inplace=True)

In [121]:
marketstack_historical_data.shape, yfinance_historical_data.shape, bme_historical_data.shape

((7604, 169), (6097, 169), (5159, 88))

In [122]:
marketstack_historical_data.isnull().sum().sum(), yfinance_historical_data.isnull().sum().sum(), bme_historical_data.isnull().sum().sum()

(1120386, 879531, 279106)

In [146]:
sub = 'BTO'

for ticker in bme_historical_data.columns:
    if sub in ticker:
        print(ticker)

BTO_1
BTO_0


In [124]:
bme_historical_data.columns

Index(['SAN', 'BTO_1', 'BKIA_0', 'BKIA', 'NTGY', 'TRE', 'ACX_0', 'ACX', 'FER',
       'ACS', 'ELE_0', 'ELE', 'SAB', 'CRI', 'IBR', 'EBRO_0', 'EBRO_1', 'AMS',
       'AENA', 'ITX', 'MTS', 'IAG', 'VIS_0', 'VIS', 'IBLA', 'BBVA', 'MAP',
       'BME', 'CIE', 'MAS', 'IBE', 'ABG', 'ENG', 'SCYR_1', 'REP', 'ABE',
       'ABG.P_0', 'ABG.P_1', 'ANA_0', 'ANA', 'EVA', 'GRF', 'CLNX', 'MRL',
       'BKT', 'POP', 'ENC', 'IDR', 'SGRE_0', 'SGRE', 'MEL', 'TL5', 'REE',
       'COL', 'JAZ', 'TEF', 'FCC', 'CABK', 'DIA', 'ALM', 'REE_0', 'CIN', 'LOR',
       'UNF', 'ZEL', 'NHH_0', 'NHH_1', 'MAP_0', 'ALT', 'BTO_0', 'FAD', 'DRC',
       'PRS_0', 'PRS_1', 'SCYR_0', 'TEM', 'CAR', 'AGS', 'COL_0', 'A3TV', 'SGC',
       'ANE', 'FDR', 'PHM', 'ROVI', 'SCYR', 'SLR', 'UNI'],
      dtype='object')

In [150]:
ibex_historical_data['BTO_1'].dropna()

date
2004-01-02    9.6000
2004-01-05    9.6800
2004-01-07    9.8000
2004-01-08    9.9600
2004-01-09    9.9700
               ...  
2010-12-27    5.8275
2010-12-28    5.8360
2010-12-29    5.7621
2010-12-30    5.8089
2011-01-03    5.8360
Name: BTO_1, Length: 1492, dtype: float64

In [145]:
marketstack_historical_data['BTO_1'].dropna()

Series([], Name: BTO_1, dtype: float64)

In [152]:
bme_historical_data['BTO_1'].dropna()

2006-08-01    10.7300
2006-08-02    10.8100
2006-08-03    10.7900
2006-08-04    10.8600
2006-08-07    10.7500
               ...   
2010-12-27     5.8275
2010-12-28     5.8360
2010-12-29     5.7621
2010-12-30     5.8089
2011-01-03     5.8360
Name: BTO_1, Length: 1125, dtype: float64

In [153]:
bme_historical_data = bme_historical_data.rename(columns={
    'ACX': 'ACX_1',
    'ACX_0': 'ACX',
    'BKIA': 'BKIA_1',
    'BKIA_0': 'BKIA',
    'ABG.P_0':'ABG.P',
    'VIS':'VIS_3',
    'VIS_0':'VIS_2',
    'EBRO_0': 'EBRO',
    'BTO_1':'BTO_2',
    'BTO_0':'BTO_1',
    'SGRE_0': 'SGRE',
    'ANA': 'ANA_1',
    'ANA_0': 'ANA',
    'REE_0': 'REE',
    'COL_0': 'COL',
    'COL': 'COL_1',
    'MAP':'MAP_2',
    'MAP_0':'MAP_1',
    'PRS_0':'PRS',

    })

In [154]:
ibex_historical_data = pd.DataFrame(columns=marketstack_historical_data.columns)

In [155]:
for c in ibex_historical_data.columns:
    try:
        ibex_historical_data[c] = marketstack_historical_data[c].fillna(yfinance_historical_data[c])
        ibex_historical_data[c] = ibex_historical_data[c].fillna(bme_historical_data[c])
        print('ok ' + c)
    except:
        print('failed ' + c)

ok A3TV
ok ABE
ok ABG.P
ok ABG.P_1
ok ABG
failed ACE
failed ACR
ok ACS
failed ACX
failed ACX_1
failed ACX_2
ok AENA
failed AGR
ok AGS
failed AGS_1
failed AGS_2
failed AGS_3
failed ALB
failed ALB_1
failed ALB_2
failed ALB_3
ok ALM
ok ALT
failed AMP
ok AMS
failed AMS_1
failed ANA
failed ANA_1
failed ANA_2
ok ANE
failed ARA
failed ARG
failed ASL
failed AUM
failed AZC
failed AZC_1
failed BBV
ok BBVA
failed BCH
failed BKIA
failed BKIA_1
ok BKT
ok BME
failed BTO
ok BTO_1
ok BTO_2
ok CABK
failed CAN
ok CAR
failed CEN
failed CEP
failed CEP_1
ok CIE
ok CIN
ok CLNX
failed COL
failed COL_1
failed CRF
ok CRI
failed CRI_1
failed CRI_2
failed CTE
failed CTF
failed CTG
failed CUB
ok DIA
ok DRC
failed EBA
ok EBRO
ok EBRO_1
failed ECR
ok ELE
failed ELE_1
ok ENC
failed ENC_1
ok ENG
ok EVA
failed EXT
ok FAD
ok FCC
ok FDR
failed FEC
ok FER
failed FOC
failed GAM
failed GAM_1
failed GAS
failed GES
failed GES_1
failed GPP
ok GRF
failed HHU
failed HHU_1
failed HID
failed HIS
ok IAG
ok IBE
ok IBLA
ok IBR
ok ID

In [ ]:
ibex_historical_data['benchmark'] = yf.download('^IBEX')['Adj Close']

In [217]:
ibex_historical_data.to_csv('ibex_historical_data.csv',index=True)

In [168]:
failed_loads = ibex_historical_data.columns[ibex_historical_data.isnull().values.all(axis=0)]
print('Missing tickers are {}'.format(len(failed_loads)))
print(np.array(failed_loads))

Missing tickers are 64
['ACE' 'ACR' 'ACX' 'AGR' 'AGS_1' 'AGS_2' 'AGS_3' 'ALB' 'ALB_1' 'ARA' 'ARG'
 'ASL' 'AUM' 'AZC' 'AZC_1' 'BBV' 'BCH' 'BTO' 'CAN' 'CEN' 'CEP' 'CEP_1'
 'CRF' 'CRI_1' 'CRI_2' 'CTE' 'CTF' 'CTG' 'CUB' 'EBA' 'ECR' 'EXT' 'FEC'
 'FOC' 'GAS' 'GES' 'GES_1' 'GPP' 'HHU' 'HHU_1' 'HID' 'HIS' 'MVC' 'MVC_1'
 'PMD' 'PRY' 'PUL' 'RAD' 'SAR' 'SCH' 'SEV' 'SOL' 'SYV' 'SYV_1' 'TAB' 'TPI'
 'TPZ' 'TRR' 'TRR_1' 'URA' 'URB' 'VAL' 'VAL_1' 'VDR']


In [215]:
for c in ibex_historical_data.columns:
    if ibex_historical_data[c].isnull().sum() != ibex_historical_data[c].shape[0]:
        first_valid = ibex_historical_data.loc[ibex_historical_data[c].notnull()][c].index[0]
        last_valid = ibex_historical_data.loc[ibex_historical_data[c].notnull()][c].index[-1]
        data = ibex_historical_data[c].loc[first_valid:last_valid]
        ibex_historical_data[c] = data.fillna(method='ffill')
        print(c, ibex_historical_data[c].loc[first_valid:last_valid].isnull().sum())
    else:
        print(c, 'null')
    

A3TV 0
ABE 0
ABG.P 0
ABG.P_1 0
ABG 0
ACE null
ACR null
ACS 0
ACX null
ACX_1 0
ACX_2 0
AENA 0
AGR null
AGS 0
AGS_1 null
AGS_2 null
AGS_3 null
ALB null
ALB_1 null
ALB_2 0
ALB_3 0
ALM 0
ALT 0
AMP 0
AMS 0
AMS_1 0
ANA 0
ANA_1 0
ANA_2 0
ANE 0
ARA null
ARG null
ASL null
AUM null
AZC null
AZC_1 null
BBV null
BBVA 0
BCH null
BKIA 0
BKIA_1 0
BKT 0
BME 0
BTO null
BTO_1 0
BTO_2 0
CABK 0
CAN null
CAR 0
CEN null
CEP null
CEP_1 null
CIE 0
CIN 0
CLNX 0
COL 0
COL_1 0
CRF null
CRI 0
CRI_1 null
CRI_2 null
CTE null
CTF null
CTG null
CUB null
DIA 0
DRC 0
EBA null
EBRO 0
EBRO_1 0
ECR null
ELE 0
ELE_1 0
ENC 0
ENC_1 0
ENG 0
EVA 0
EXT null
FAD 0
FCC 0
FDR 0
FEC null
FER 0
FOC null
GAM 0
GAM_1 0
GAS null
GES null
GES_1 null
GPP null
GRF 0
HHU null
HHU_1 null
HID null
HIS null
IAG 0
IBE 0
IBLA 0
IBR 0
IDR 0
ITX 0
JAZ 0
LOG 0
LOR 0
MAP 0
MAP_1 0
MAP_2 0
MAS 0
MEL 0
MRL 0
MTS 0
MVC null
MVC_1 null
NHH 0
NHH_1 0
NHH_2 0
NTGY 0
OHLA 0
PHM 0
PMD null
POP 0
PRS 0
PRS_1 0
PRY null
PUL null
RAD null
RED 0
REE 0
REE_1 0


In [218]:
returns = np.log(ibex_historical_data).diff()[1:]
returns.to_csv('ibex_historical_returns.csv',index=True)